# Lightweight Python components

To build a component just define a stand-alone python function and then call `kfp.components.func_to_container_op(func)` to convert it to a component that can be used in a pipeline.

There are several requirements for the function:

- The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
- The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package.)
- If the function operates on numbers, the parameters need to have type hints. Supported types are [int, float, bool]. Everything else is passed as string.
- To build a component with multiple output values, use the typing.NamedTuple type hint syntax: `NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])`

In [2]:
import kfp.components as comp

## Start with a simple function

In [3]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation

In [4]:
add_op = comp.func_to_container_op(add)

In [5]:
type(add_op)

function

## A more complicate function with multiple outputs

In [6]:
#Advanced function
#Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple
def my_divmod(dividend: float, 
              divisor:float, 
              output_dir:str = './') -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    '''Divides two numbers and calculate  the quotient and remainder'''
    
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open(output_dir + 'mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

In [7]:
my_divmod(100, 7)

MyDivmodOutput(quotient=14, remainder=2)

In [8]:
divmod_op = comp.func_to_container_op(my_divmod)

## Define the pipeline

In [9]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='a',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b, '/')

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c)

## Compile the pipeline

In [10]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [11]:
! ls .

00. Setup.ipynb
01. Local Development Quickstart.ipynb
02. Lightweight Python components.ipynb
03. GCP Components Pipeline.ipynb
04. TFX Pipeline from Pre-build Components.ipynb
05. Spark and XGboost Pipeline.ipynb
calc_pipeline.pipeline.zip
mlpipeline-metrics.json
mlpipeline-ui-metadata.json


## Submit the pipeline

**If submit outside the kubeflow cluster, need the following**
- host = "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- And, you'll first need to create OAuth client ID credentials of type `Other` according to the tutorial [here](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

In [27]:
host = "https://kubeflow1.endpoints.kubeflow-pipeline-fantasy.cloud.goog/pipeline"
client_id = "493831447550-os23o55235htd9v45a9lsejv8d1plhd0.apps.googleusercontent.com"
other_client_id = "493831447550-iu24vv6id3ng5smhf2lboovv5qukuhbh.apps.googleusercontent.com"
other_client_secret = "cB8Xj-rb9JWCYcCRDlpTMfhc"

**If you run and submit within the kubeflow cluster**, the following is enough
```python
client = kfp.Client()
```

In [28]:
#Get or create an experiment and submit a pipeline run
import kfp

import kubernetes as k8s
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    client = kfp.Client(host=host, 
                        client_id=client_id,
                        other_client_id=other_client_id, 
                        other_client_secret=other_client_secret)

https://accounts.google.com/o/oauth2/v2/auth?client_id=493831447550-iu24vv6id3ng5smhf2lboovv5qukuhbh.apps.googleusercontent.com&response_type=code&scope=openid%20email&access_type=offline&redirect_uri=urn:ietf:wg:oauth:2.0:oob
Authorization code: 4/twFw3g2PGvm37EAksT2mrIzT1_Dn-7T_QBCxUXhLkWaabsTNAlQdPIg


In [29]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

experiment = client.create_experiment('python-functions')

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

False